1\. Write a function that converts number representation (bin<->dec<->hex)

In [17]:
def bd (string):
    x = [int(i) for i in string]
    x.reverse()
    y = int(sum(x[i]*pow(2,i) for i in range(0,len(x)-1)))
    if x[len(x)-1] == 1:
        bd= -y
    else: bd = y
    return bd
    
bd("10010111")

def db (number):
    b = []
    a = number
    while a != 0:
        if a%2 == 0:
            b.append(0)
            a //= 2
        else:
            b.append(1)
            a //= 2
    b.reverse()
    if number > 0:
        b.insert(0,0)
    else: b.insert(0,1) 
    g = ''.join(map(str,b))
    print(''.join(map(str,b)))
    return g
    
db(23)

def dh(number):
    d = {10:'A',11:'B',12:'C',13:'D',14:'E',15:'F'}
    x=[]
    a = number
    while a != 0:
        if a%16 <= 9:
            x.append(a%16)
            a //= 16
        else:
            x.append(d[a%16])
            a //= 16
    x.reverse()
    g = ''.join(map(str,x))
    print(g)
    return g

dh(520899)

def hd(hexa):
    d = {'A':10,'B':11,'C':12,'D':13,'E':14,'F':15}
    x=[]
    a = [int(ele) if ele.isdigit() else ele for ele in hexa]
    for i in range(0,len(a)):
        if type(a[i]) is int:
            x.append(a[i]*pow(16,(len(a)-i-1)))
        else: 
            x.append(d[a[i]]*pow(16,(len(a)-i-1)))
    print(int(sum(x)))
    return int(sum(x))
hd('7F2C3')

def bh(string):
    return dh(bd(string))
bh("00011001")

def hb(string):
    return db(hd(string))
hb('1F')

x = input('From wich number representation (binary,hexa,decimal):')
y = input('To wich number representation (binary,hexa,decimal):')
number = input('Insert the number:')
def Convertion(x,y,number):
    if x == 'binary' and y == 'hexa':
        print("The convertion from {} to {} is:".format(x,y), bh(str(number)))
    elif x == 'binary' and y == 'decimal':
        print("The convertion from {} to {} is:".format(x,y), bd(str(number)))
    elif x == 'decimal' and y == 'hexa':
        print("The convertion from {} to {} is:".format(x,y), dh(int(number)))
    elif x == 'decimal' and y == 'binary':
        print("The convertion from {} to {} is:".format(x,y), db(int(number)))
    elif x == 'hexa' and y == 'decimal':
        print("The convertion from {} to {} is:".format(x,y), hd(str(number)))
    elif x == 'hexa' and y == 'binary':
        print("The convertion from {} to {} is:".format(x,y), hb(str(number)))
Convertion(x,y,number)

010111
7F2C3
520899
19
31
011111
From wich number representation (binary,hexa,decimal):hexa
To wich number representation (binary,hexa,decimal):binary
Insert the number:1F
31
011111
The convertion from hexa to binary is: 011111


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [23]:
a = ('11100101000101000010101000011000')

def bd (string):
    x = [int(i) for i in string]
    x.reverse()
    y = int(sum(x[i]*pow(2,i) for i in range(0,len(x))))
    return y

def conv32(a):
    s = int(a[0])
    print("s = ",s)
    bias = 127
    e = bd(a[1:9])
    print("e = ",e)
    f = sum(int(a[40-i])*pow(2,-(32-i)) for i in range(9,32))
    print("f = ",f)
    num = pow(-1,s)*(1+f)*pow(2,e-bias)
    return num
 
conv32(a)

s =  1
e =  202
f =  0.15753459930419922


-4.373042075612876e+22

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [8]:
#sistemare printando iterazione e risultato
import math
under = 1.0
over = 1.0
d = 2
e = 1200
o=0
u=0
ov = []
un = []
for i in range(e):
    while over != float("inf"):
        over *=2
        o+=1
        ov.append(over)
        #print(o, "\t", over, "\t")


for a in range(e):
    while under != 0.0:
        under /=2
        u+=1
        un.append(under)
        #print(u, "\t", under, "\t")
print("After {} interaction overflow is reached.".format(o))
print(ov[-2])
print("After {} interaction underflow is reached.".format(u))
print(un[-2])

After 1024 interaction overflow is reached.
8.98846567431158e+307
After 1075 interaction underflow is reached.
5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [1]:
i1=0
a=1.0
f=1.0/2
b=a+f

while a!=b:
    a=b
    f/=2
    b+=f
    i1+=1
    

#print(i1,a,b)
print("Machine precision is :", 2*2**-i1)

Machine precision is : 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [2]:
import numpy as np
def sol(a,b,c):
    x1 = (-b + np.sqrt(pow(b,2)-4*a*c))/(2*a) 
    x2 = (-b - np.sqrt(pow(b,2)-4*a*c))/(2*a)
    #print(x1,x2)
    return x1,x2
    
print("Solutions of the equation (x1,x2) = ", sol(0.001,1000,0.001))

Solutions of the equation (x1,x2) =  (-9.999894245993346e-07, -999999.999999)


In [6]:
def sol(a,b,c):
    x1 = ((-b + np.sqrt(pow(b,2)-4*a*c))*(-b - np.sqrt(pow(b,2)-4*a*c)))/((2*a)*(-b - np.sqrt(pow(b,2)-4*a*c))) 
    x2 = ((-b - np.sqrt(pow(b,2)-4*a*c))*(-b + np.sqrt(pow(b,2)-4*a*c)))/((2*a)*(-b + np.sqrt(pow(b,2)-4*a*c)))
    den = ((2*a)*(-b + np.sqrt(pow(b,2)-4*a*c)))
    num = ((-b - np.sqrt(pow(b,2)-4*a*c))*(-b + np.sqrt(pow(b,2)-4*a*c)))
    #print(x1,x2)
    print("x2 numerator is:",num,"\nx2 denominator is: ", den)
    return x1,x2
    
print("Solutions of the equation (x1,x2) = ", sol(0.001,1000,0.001))

x2 numerator is: 3.999957698393339e-06 
x2 denominator is:  -3.999957698397338e-12
Solutions of the equation (x1,x2) =  (-9.999894245993346e-07, -999999.9999990001)


We have more decimal numbers due to new operation. In particular, x2 has more digits after the decimal point. This is caused by the fact that in $\sqrt{b^2-4ac}$ we are subtracting a very small number (4ac) from $b^2$. Therefore the denominator of x2 $2a(-b-\sqrt{b^2-4ac})$ is very close to zero and since the computer saves the data in binary format, unexpected digits appear after the comma during the operations (it can be noticed by sequencing all operations). 
In x1 the result does not change because instead the denominator is not close to zero and therefore no brutal operations are introduced.

In [14]:
import math
#math.copysign(1, b)
def roots(a,b,c):
    root = pow(b,2)-4*a*c
    if root>0:
        x1 = (-b - math.copysign(1, b) * np.sqrt(root))/(2*a) 
        x2 = c/(a*x1)
        print("There are two roots: x1 = ",x1,"; x2 = ",x2)
    elif root == 0:
        x12 = - b/(2*a)
        print("There is a root: x12 = ", x12)
    else:
        x1 =complex(- b/(2*a), (np.sqrt(-pow(b,2)+4*a*c))/(2*a))
        x2 =complex(- b/(2*a), (np.sqrt(-pow(b,2)+4*a*c))/(2*a))
        print("Roots are complex",x1,x2)

roots(0.001,1000,0.001)

There are two roots: x1 =  -999999.999999 ; x2 =  -1.000000000001e-06


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [18]:
def derivative(x,delta):
    f1 = (x+delta)*(x+delta-1)
    #print(x+delta,x+delta-1)
    f2 = x*(x-1)
    ft = (f1-f2)/delta
    return ft

print("Derivative of the function at the point  𝑥=1 :",derivative(1,pow(10,-2)))

Derivative of the function at the point  𝑥=1 : 1.010000000000001


Since the difference (x + delta-1) = 1 + delta-1 is very close and the computer saves the numbers in binary format ft = (f1-f2) / delta approximates the last digit 8 to 1 that's why you see 1 at the end. Analyzing and decomposing x + delta and multiplying by x + delta-1 gives us an unexpected 8. Finally, by analyzing all the operations we also notice an unexpected 9 which, as we said, is attributable to the store of the numbers by the computer.

In [19]:
def f(x):
    return x*(x-1)

def derivative(x,delta):
    return (f(x+delta)-f(x))/delta
    
derivative(1,pow(10,-2))

1.010000000000001

In [23]:
for i in [4,6,8,10,12,14]:
    print("Delta = ",pow(10,-i),"Derivative: ",derivative(1,pow(10,-i)))
    #print("a=",derivative(1,pow(10,-i+1))/derivative(1,pow(10,-i)))

Delta =  0.0001 Derivative:  1.0000999999998899
Delta =  1e-06 Derivative:  1.0000009999177333
Delta =  1e-08 Derivative:  1.0000000039225287
Delta =  1e-10 Derivative:  1.000000082840371
Delta =  1e-12 Derivative:  1.0000889005833413
Delta =  1e-14 Derivative:  0.9992007221626509


The result would seem to improve as the delta decreases. However, the last two results there is an unexpected worsening of the value of the derivative.

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [59]:
#(a) Write a programe to compute the integral with  𝑁=100 . How does the result compares to the true value?
import numpy as np
import math

N=100
h = 2./N
integral = 0

def f(x):
    return sqrt(1-pow(x,2))

for i in range(1,N+1):
    a=-1
    integral += f(a+(h*i))*h

print("The computed value is: ",integral)
print("The difference between the true and computed value is: ", math.pi/2 - integral)

The computed value is:  1.5691342555492505
The difference between the true and computed value is:  0.0016620712456461018


In [69]:
#(b) How much can  𝑁  be increased if the computation needs to be run in less than a second? 


import numpy as np
import math
import time

N=1

def f(x):
    return sqrt(1-pow(x,2))

timeend = time.time() + 1 
while time.time() <= timeend:
    integral=0
    for i in range(1,N+1):
        a=-1
        integral+=f(a+((2/N)*i))*(2/N)
    N+=1
    
print("N:",N,"\nIntegral value is:",integral, "\nDifference between real value: ",math.pi/2 - integral )

N: 1181 
Integral value is: 1.5707546123368306 
Difference between real value:  4.1027034841656373e-05


In [72]:
#What is the gain in running it for 1 minute?
import numpy as np
import math
import time

N=1

def f(x):
    return sqrt(1-pow(x,2))

timeend = time.time() + 60 
while time.time() <= timeend:
    integral2=0
    for i in range(1,N+1):
        a=-1
        integral2+=f(a+((2/N)*i))*(2/N)
    N+=1
    
print("N:",N,"\nIntegral value is:",integral2, "\nDifference between real value: ",math.pi/2 - integral2 )

gain = np.abs((integral-integral2)/integral2)*100
print("The gain is:",gain,"%")

N: 9137 
Integral value is: 1.570794422305105 
Difference between real value:  1.9044897916487002e-06
The gain is: 0.002510669272087587 %
